In [35]:
import os
import pandas as pd
import numpy as np
import openai
import tiktoken

API_KEY_ENV = "LLM_DEMO_KEY"
API_ORG_ENV = "CENTRIC_OPENAI_ORG"
MODEL = 'gpt-3.5-turbo'
pd.set_option('display.max_colwidth', None)

openai.api_key = os.getenv(API_KEY_ENV)
openai.organization = os.getenv(API_ORG_ENV)
encoding = tiktoken.encoding_for_model(MODEL)

In [36]:
file = os.path.join("data", "symptom_data.csv")
types = {
    'label': 'string',
    'text': 'string'
}
df = pd.read_csv(file, index_col = 0, dtype=types)
df.shape

(1200, 2)

In [37]:
df.head()

,label,text
0,Psoriasis,"I have been experiencing a skin rash on my arms, legs, and torso for the past few weeks. It is red, itchy, and covered in dry, scaly patches."
1,Psoriasis,"My skin has been peeling, especially on my knees, elbows, and scalp. This peeling is often accompanied by a burning or stinging sensation."
2,Psoriasis,"I have been experiencing joint pain in my fingers, wrists, and knees. The pain is often achy and throbbing, and it gets worse when I move my joints."
3,Psoriasis,"There is a silver like dusting on my skin, especially on my lower back and scalp. This dusting is made up of small scales that flake off easily when I scratch them."
4,Psoriasis,"My nails have small dents or pits in them, and they often feel inflammatory and tender to the touch. Even there are minor rashes on my arms."


In [38]:
# Count characters in Plot column
df['text_characters'] = df['text'].apply(len)
# Count tokens in Plot Column
df['text_token_count'] = df['text'].apply(lambda plot: len(encoding.encode(plot)))
df.head()

,label,text,text_characters,text_token_count
0,Psoriasis,"I have been experiencing a skin rash on my arms, legs, and torso for the past few weeks. It is red, itchy, and covered in dry, scaly patches.",141,37
1,Psoriasis,"My skin has been peeling, especially on my knees, elbows, and scalp. This peeling is often accompanied by a burning or stinging sensation.",138,31
2,Psoriasis,"I have been experiencing joint pain in my fingers, wrists, and knees. The pain is often achy and throbbing, and it gets worse when I move my joints.",148,35
3,Psoriasis,"There is a silver like dusting on my skin, especially on my lower back and scalp. This dusting is made up of small scales that flake off easily when I scratch them.",164,38
4,Psoriasis,"My nails have small dents or pits in them, and they often feel inflammatory and tender to the touch. Even there are minor rashes on my arms.",140,32


In [39]:
max(df['text_token_count'])

69

In [34]:
df.label.value_counts()

label
Bronchial Asthma                   50
Common Cold                        50
Migraine                           50
Pneumonia                          50
Typhoid                            50
Fungal infection                   50
Dengue                             50
Dimorphic Hemorrhoids              50
Varicose Veins                     50
Impetigo                           50
Chicken pox                        50
Malaria                            50
Hypertension                       50
urinary tract infection            50
Psoriasis                          50
Jaundice                           50
drug reaction                      50
Cervical spondylosis               50
Arthritis                          50
Acne                               50
allergy                            50
gastroesophageal reflux disease    50
peptic ulcer disease               50
diabetes                           50
Name: count, dtype: Int64

In [12]:
df = df.query('text_token_count <= 4000') # just in case, not necessary with this set
df.sort_values(by = ['text_token_count'], ascending = False, inplace=True)
df.head()

,label,text,text_characters,text_token_count
18,Bronchial Asthma,I've had this cough that's been lingering for ...,302,69
19,Bronchial Asthma,"I've had a persistent cough for days, and I al...",278,65
32,Bronchial Asthma,must confess that I've been experiencing shor...,283,65
69,Common Cold,"My eyes are always red and itchy, and my nose ...",264,65
70,Common Cold,"My nose always feels stuffy and congested, and...",257,64


In [40]:
sample = df.sample(n=1).iloc[0,:]

summary = openai.ChatCompletion.create(model = MODEL, 
                                      messages = [{'role':'user', 
                                                   'content': "using 4 words or less please summarize these symptoms: {}".format(sample.text)
                                                  }])

predicted_diag = openai.ChatCompletion.create(model = MODEL, 
                                      messages = [{'role':'user', 
                                                   'content': "what's your best guess at this person's condition?: {}".format(sample.text)
                                                  }])

print("Patient Description\n============\n{}".format(sample.text))
print("\n\nLLM Summary\n============\n{}".format(summary.choices[0].message.content))
print("\n\nActual Label\n============\n{}".format(sample.label))
print("\n\nPredicted Diagnosis\n============\n{}".format(predicted_diag.choices[0].message.content))

Patient Description
I've been feeling really sick with a high fever, shortness of breath, sweating, chills, and extreme fatigue. My heart rate is rapid, and I've been coughing up a lot of brownish sputum.


LLM Summary
Severe respiratory illness, haziar answe


Actual Label
Pneumonia


Predicted Diagnosis
I'm not a doctor, but based on the symptoms you mentioned, this person could potentially have a respiratory infection such as bronchitis or pneumonia. The presence of a high fever, shortness of breath, sweating, chills, and fatigue may indicate a significant illness. The coughing up of brownish sputum may suggest the presence of blood or other concerning factors, which warrants immediate medical attention. It is essential for this person to consult a healthcare professional for an accurate diagnosis and appropriate treatment.
